In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set()

In [ ]:
df = pd.read_csv('../data/simple_trig.csv')

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
sample = df.sample(3, random_state=8)
labels = sample['labels'].tolist()
series_samples = sample[df.columns[1:]]

In [ ]:
def plot_random_series(df, n_series):
    
    sample = df.sample(n_series, random_state=8)
    labels = sample['labels'].tolist()
    series_samples = sample[df.columns[1:]]
    
    plt.figure(figsize=(10,6))
    
    for i in range(series_samples.shape[0]):
        pd.Series(series_samples.iloc[i]).astype(np.float64).plot(linewidth=1.5)
    
    plt.title('Randomly Selected Data')
    plt.legend(labels)
    
plot_random_series(df, 2)

In [ ]:
pred_steps = 14

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv1D, Dense, Dropout, Lambda, concatenate
from keras.optimizers import Adam

# convolutional layer parameters
n_filters = 32 
filter_width = 2
# 6 is good for 512 input length
dilation_rates = [2**i for i in range(6)] 

# define an input history series and pass it through a stack of dilated causal convolutions. 
history_seq = Input(shape=(None, 1))
x = history_seq

for dilation_rate in dilation_rates:
    x = Conv1D(filters=n_filters,
               kernel_size=filter_width, 
               padding='causal',
               dilation_rate=dilation_rate)(x)

x = Dense(128, activation='relu')(x)
x = Dropout(.2)(x)
x = Dense(1)(x)

# extract the last 14 time steps as the training target
def slice(x, seq_length):
    return x[:,-seq_length:,:]

pred_seq_train = Lambda(slice, arguments={'seq_length':14})(x)

model = Model(history_seq, pred_seq_train)

In [ ]:
model.summary()

In [ ]:
first_n_samples = 40000
batch_size = 2**11
epochs = 100

all_series = df[df.columns[1:]].values
series_mean = all_series.mean(axis=1).reshape(-1,1)
all_series = all_series - series_mean

encoder_input_data=all_series[:first_n_samples,:512].reshape(first_n_samples,-1,1)
decoder_target_data=all_series[:first_n_samples,512+1-pred_steps:512+1].reshape(first_n_samples,-1,1)

print("encoder_input_data.shape=", encoder_input_data.shape)
print("decoder_target_data.shape=", decoder_target_data.shape)

In [ ]:
model.compile(Adam(), loss='mean_absolute_error')
history = model.fit(encoder_input_data, decoder_target_data,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_split=0.2)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.xlabel('Epoch')
plt.ylabel('Mean Absolute Error Loss')
plt.title('Loss Over Time')
plt.legend(['Train','Valid'])

In [ ]:
def predict_sequence(input_sequence):

    history_sequence = input_sequence.copy()
    pred_sequence = np.zeros((1,pred_steps,1)) # initialize output (pred_steps time steps)  
    
    for i in range(pred_steps):
        
        # record next time step prediction (last time step of model output) 
        last_step_pred = model.predict(history_sequence)[0,-1,0]
        pred_sequence[0,i,0] = last_step_pred
        
        # add the next time step prediction to the history sequence
        history_sequence = np.concatenate([history_sequence, 
                                           last_step_pred.reshape(-1,1,1)], axis=1)

    return pred_sequence

In [ ]:
left_samples = all_series.shape[0] - first_n_samples
left_samples

In [ ]:
# 512 points, [0]-[511]
test_input_data=all_series[first_n_samples:,:512].reshape(-1,512,1)
# pred_steps points, [512]-[526]
test_target_data=all_series[first_n_samples:,512:512+pred_steps].reshape(-1,pred_steps,1)

print("test_input_data.shape=", test_input_data.shape)
print("test_target_data.shape=", test_target_data.shape)

In [ ]:
def predict_and_plot(input_data, target_data, sample_ind, input_disp_len=50):
    input_series = input_data[sample_ind,:,0].reshape(1,-1,1)
    pred_series  = predict_sequence(input_series)
    
    input_series  = input_series.reshape(-1)
    pred_series   = pred_series.reshape(-1)
    target_series = target_data[sample_ind,:,:].reshape(-1)
    
    input_series_tail  = input_series[-input_disp_len:]
    plt.figure(figsize =(10,6))   
    
    tail_range   = range(512-input_disp_len,512)
    target_range = range(512+1,512+1+pred_steps)
    pred_range   = target_range
    plt.plot(tail_range,input_series_tail,linewidth=3)
    plt.plot(target_range,target_series,color='orange')
    plt.plot(target_range,pred_series,color='teal',linestyle='--')
    
    plt.title('Encoder Series Tail of Length %d, Target Series, and Predictions' % input_disp_len)
    plt.legend(['Encoding Series','Target Series','Predictions'])

In [ ]:
predict_and_plot(test_input_data, test_target_data, 100)

In [ ]:
predict_and_plot(test_input_data, test_target_data, 6007)

In [ ]:
predict_and_plot(test_input_data, test_target_data, 10000)

In [ ]:
predict_and_plot(test_input_data, test_target_data, 15000)

In [ ]:
predict_and_plot(test_input_data, test_target_data, 25000)

In [ ]:
model.save("simple_trig_model")